# 단일 에이전트 실습 — ReAct 원리 + 더미 MCP 연동 

이 노트북은 **단일 Agent의 기본 루프(Thought → Action → Observation)** 를 구현하고,
더미 **MCP 서버**와의 통신을 통해 **툴 호출을 표준화된 방식**으로 수행하는 실습 자료입니다.


## 0) 설치 & 기본 설정

Colab에서 아래를 실행하세요. (이 문서 내부 실행 순서대로 진행)

In [2]:

!pip -q install fastapi uvicorn jsonschema requests pandas tabulate 
!pip -q install smolagents 

import json, time, threading
from typing import Dict, Any, List
from pprint import pprint
import requests
from jsonschema import Draft202012Validator, ValidationError
import pandas as pd


## 1) 수업용 금융 코퍼스 

In [3]:
FINANCE_DOCS = [
    {"doc_id":"d1","text":"삼성전자 메모리 부문은 DDR5 전환과 HBM 수요 확대에 힘입어 수익성이 회복되고 있다."},
    {"doc_id":"d2","text":"SK하이닉스는 HBM3E 양산을 통해 엔비디아와의 파트너십을 강화하고 매출 다변화를 꾀한다."},
    {"doc_id":"d3","text":"엔비디아는 데이터센터용 GPU 출하가 확대되어 분기 최대 실적을 갱신했다."},
    {"doc_id":"d4","text":"TSMC는 3나노 공정 수율 개선으로 하이엔드 고객 주문을 확보하며 파운드리 점유율을 높였다."},
    {"doc_id":"d5","text":"애플은 A시리즈 칩 자체 설계를 강화하고 TSMC와의 공조로 성능과 전력 효율을 개선한다."},
    {"doc_id":"d6","text":"미국의 기준금리 동결 기조가 지속될 경우 성장주 밸류에이션은 점진적 회복세를 보일 수 있다."},
    {"doc_id":"d7","text":"원/달러 환율 상승은 반도체 수출 채산성에 단기적으로 긍정적 영향을 준다."},
    {"doc_id":"d8","text":"HBM 채택 확대는 LLM 추론 효율과 메모리 대역폭 병목을 동시에 개선한다."},
    {"doc_id":"d9","text":"반도체 공급망 다변화는 지정학적 리스크를 관리하는 핵심 전략으로 자리잡고 있다."},
    {"doc_id":"d10","text":"서버 DRAM 수요는 AI 워크로드 증가와 함께 중장기 성장 추세가 지속될 전망이다."},
]
pd.DataFrame(FINANCE_DOCS)


,doc_id,text
0,d1,삼성전자 메모리 부문은 DDR5 전환과 HBM 수요 확대에 힘입어 수익성이 회복되고...
1,d2,SK하이닉스는 HBM3E 양산을 통해 엔비디아와의 파트너십을 강화하고 매출 다변화를...
2,d3,엔비디아는 데이터센터용 GPU 출하가 확대되어 분기 최대 실적을 갱신했다.
3,d4,TSMC는 3나노 공정 수율 개선으로 하이엔드 고객 주문을 확보하며 파운드리 점유율...
4,d5,애플은 A시리즈 칩 자체 설계를 강화하고 TSMC와의 공조로 성능과 전력 효율을 개...
5,d6,미국의 기준금리 동결 기조가 지속될 경우 성장주 밸류에이션은 점진적 회복세를 보일 ...
6,d7,원/달러 환율 상승은 반도체 수출 채산성에 단기적으로 긍정적 영향을 준다.
7,d8,HBM 채택 확대는 LLM 추론 효율과 메모리 대역폭 병목을 동시에 개선한다.
8,d9,반도체 공급망 다변화는 지정학적 리스크를 관리하는 핵심 전략으로 자리잡고 있다.
9,d10,서버 DRAM 수요는 AI 워크로드 증가와 함께 중장기 성장 추세가 지속될 전망이다.


## 2) 더미 MCP Manifest

In [4]:
MANIFEST = {
  "modelContextProtocolVersion": "2025-11-01",
  "server": {"name": "dummy-mcp", "version": "1.0.0"},
  "tools": [
    {
      "name": "calc_add",
      "description": "두 수를 더합니다.",
      "inputSchema": {
        "type": "object",
        "properties": {
          "a": {"type": "number"},
          "b": {"type": "number"}
        },
        "required": ["a","b"],
        "additionalProperties": False
      },
      "outputSchema": {
        "type": "object",
        "properties": {"result": {"type": "number"}},
        "required": ["result"]
      }
    },
    {
      "name": "fx_convert",
      "description": "USD를 KRW로 환산 (고정 환율: 1 USD = 1350 KRW)",
      "inputSchema": {
        "type": "object",
        "properties": {"amount_usd": {"type": "number", "minimum": 0}},
        "required": ["amount_usd"],
        "additionalProperties": False
      },
      "outputSchema": {
        "type": "object",
        "properties": {"amount_krw": {"type": "number"}, "rate": {"type": "number"}},
        "required": ["amount_krw","rate"]
      }
    },
    {
      "name": "kb_search",
      "description": "내장 금융 코퍼스에서 키워드 검색",
      "inputSchema": {
        "type": "object",
        "properties": {
          "query": {"type": "string", "minLength": 1},
          "top_k": {"type": "integer", "minimum": 1, "maximum": 10, "default": 3}
        },
        "required": ["query"],
        "additionalProperties": False
      },
      "outputSchema": {
        "type": "object",
        "properties": {
          "hits": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "doc_id": {"type": "string"},
                "text": {"type": "string"},
                "score": {"type": "number"}
              },
              "required": ["doc_id","text","score"]
            }
          }
        },
        "required": ["hits"]
      }
    }
  ],
  "resources": [{"name":"kb.corpus","description":"수업용 금융 코퍼스","uri":"/resources/kb.corpus"}]
}
pd.DataFrame([
  {"tool": t["name"], "desc": t["description"],
   "input_required": ",".join(t["inputSchema"].get("required", [])),
   "output_required": ",".join(t["outputSchema"].get("required", []))}
  for t in MANIFEST["tools"]
])


,tool,desc,input_required,output_required
0,calc_add,두 수를 더합니다.,"a,b",result
1,fx_convert,USD를 KRW로 환산 (고정 환율: 1 USD = 1350 KRW),amount_usd,"amount_krw,rate"
2,kb_search,내장 금융 코퍼스에서 키워드 검색,query,hits


## 3) 더미 MCP 서버 구현 (FastAPI)

In [5]:
from fastapi import FastAPI, HTTPException
import uvicorn
app = FastAPI()

@app.get('/manifest')
def get_manifest():
    return MANIFEST

@app.get('/resources/kb.corpus')
def get_corpus():
    return {"docs": FINANCE_DOCS}

from jsonschema import Draft202012Validator, ValidationError
@app.post('/tools/calc_add')
def post_calc_add(payload: dict):
    schema = next(t['inputSchema'] for t in MANIFEST['tools'] if t['name']=='calc_add')
    try:
        Draft202012Validator(schema).validate(payload)
    except ValidationError as e:
        raise HTTPException(status_code=400, detail={"error":"invalid_schema","message":str(e)})
    return {"result": float(payload['a']) + float(payload['b'])}

@app.post('/tools/fx_convert')
def post_fx_convert(payload: dict):
    schema = next(t['inputSchema'] for t in MANIFEST['tools'] if t['name']=='fx_convert')
    try:
        Draft202012Validator(schema).validate(payload)
    except ValidationError as e:
        raise HTTPException(status_code=400, detail={"error":"invalid_schema","message":str(e)})
    rate = 1350.0
    return {"amount_krw": float(payload['amount_usd'])*rate, "rate": rate}

@app.post('/tools/kb_search')
def post_kb_search(payload: dict):
    schema = next(t['inputSchema'] for t in MANIFEST['tools'] if t['name']=='kb_search')
    try:
        Draft202012Validator(schema).validate(payload)
    except ValidationError as e:
        raise HTTPException(status_code=400, detail={"error":"invalid_schema","message":str(e)})
    q = payload['query'].lower(); k = int(payload.get('top_k', 3))
    terms = [w for w in q.split() if w.strip()]
    scored = []
    for d in FINANCE_DOCS:
        s = sum(1 for t in terms if t in d['text'].lower())
        if s>0:
            scored.append({"doc_id": d['doc_id'], "text": d['text'], "score": float(s)})
    scored.sort(key=lambda x: x['score'], reverse=True)
    return {"hits": scored[:k]}

def run_server():
    uvicorn.run(app, host='127.0.0.1', port=8000, log_level='warning')

# Colab에서 실행:
import threading, time
th = threading.Thread(target=run_server, daemon=True)
th.start(); time.sleep(1)
print('Dummy MCP server running at http://127.0.0.1:8000')


Dummy MCP server running at http://127.0.0.1:8000


## 4) MCP 클라이언트 & 스키마 파악

In [6]:
import requests
BASE = 'http://127.0.0.1:8000'

def fetch_manifest(base=BASE):
    r = requests.get(f'{base}/manifest', timeout=3); r.raise_for_status(); return r.json()
    
manifest = fetch_manifest()

pd.DataFrame([{ 'tool':t['name'], 'desc':t['description']} for t in manifest['tools']])


,tool,desc
0,calc_add,두 수를 더합니다.
1,fx_convert,USD를 KRW로 환산 (고정 환율: 1 USD = 1350 KRW)
2,kb_search,내장 금융 코퍼스에서 키워드 검색


## 5) 스키마 검증 + 툴 호출 유틸

In [7]:
from jsonschema import Draft202012Validator, ValidationError

def get_tool(manifest: dict, name: str) -> dict:
    return next((t for t in manifest['tools'] if t['name']==name), None)

def validate_input(tool: dict, payload: dict):
    Draft202012Validator(tool['inputSchema']).validate(payload)

def call_tool(base: str, name: str, payload: dict) -> dict:
    r = requests.post(f'{base}/tools/{name}', json=payload, timeout=5)
    if r.status_code != 200:
        raise RuntimeError(f'HTTP {r.status_code}: {r.text}')
    return r.json()


## 6) 단일 ReAct 루프 (간단 규칙 기반)

In [8]:
import uuid, datetime, time, re

def decide_tool(question: str) -> dict:
    q = question.lower()
    if '더해' in q or '합' in q or 'sum' in q or '+' in q:
        nums = [float(x) for x in re.findall(r'[-+]?[0-9]*\.?[0-9]+', question)]
        a = nums[0] if len(nums)>0 else 0.0
        b = nums[1] if len(nums)>1 else 0.0
        return {'tool':'calc_add', 'args':{'a':a,'b':b}, 'reason':'산술 합 연산 필요'}
    if '환산' in q or '환율' in q or 'usd' in q or 'krw' in q:
        nums = [float(x) for x in re.findall(r'[-+]?[0-9]*\.?[0-9]+', question)]
        usd = nums[0] if len(nums)>0 else 1.0
        return {'tool':'fx_convert', 'args':{'amount_usd':usd}, 'reason':'USD→KRW 환산 필요'}
    return {'tool':'kb_search', 'args':{'query':question, 'top_k':3}, 'reason':'코퍼스 검색이 적절'}

def react_once(question: str, base=BASE) -> dict:
    run_id = f"{datetime.datetime.utcnow().isoformat()}Z-{uuid.uuid4().hex[:6]}"
    trace = {'run_id': run_id, 'steps': [], 'summary':{}}
    manifest = fetch_manifest(base)
    decision = decide_tool(question)
    tool = get_tool(manifest, decision['tool'])
    thought = f"질문 해석 결과: {decision['reason']} → {decision['tool']} 사용"
    t0 = time.time()
    try:
        validate_input(tool, decision['args'])
        obs = call_tool(base, decision['tool'], decision['args'])
        ok = True
    except Exception as e:
        obs = {'error': str(e)}
        ok = False
    elapsed = int((time.time()-t0)*1000)
    trace['steps'].append({'step':1,'thought':thought,'action':{'tool':decision['tool'],'args':decision['args']},'observation':obs,'elapsed_ms':elapsed})
    trace['summary'] = {'question':question,'success': bool(ok and 'error' not in obs),'tool_calls':1,'errors':0 if ok else 1,'total_elapsed_ms':elapsed}
    return trace

react_once('USD 100을 원화로 환산해줘')


{'run_id': '2025-11-13T08:30:05.896841Z-f86d3b',
 'steps': [{'step': 1,
   'thought': '질문 해석 결과: USD→KRW 환산 필요 → fx_convert 사용',
   'action': {'tool': 'fx_convert', 'args': {'amount_usd': 100.0}},
   'observation': {'amount_krw': 135000.0, 'rate': 1350.0},
   'elapsed_ms': 3}],
 'summary': {'question': 'USD 100을 원화로 환산해줘',
  'success': True,
  'tool_calls': 1,
  'errors': 0,
  'total_elapsed_ms': 3}}

## 7) Trace 저장/확인

In [9]:
import os, json
def save_trace(trace: dict, path: str = 'agent_trace.jsonl'):
    with open(path, 'a', encoding='utf-8') as f:
        f.write(json.dumps(trace, ensure_ascii=False)+'\n')
    return os.path.abspath(path)
save_trace(react_once('3+5 더해줘'))


'/workspace/agent_trace.jsonl'

## 8) SmolAgent로 따라하기

In [10]:
import os
from typing import Dict, Any

try:
    from smolagents import Tool, ToolCallingAgent, OpenAIModel
    SMOL_OK = True
except Exception as e:
    SMOL_OK = False
    print("⚠️ smolagents 임포트 실패:", e)


def call_tool(base: str, tool_name: str, payload: Dict[str, Any]):
    import requests
    url = f"{base}/tools/{tool_name}"
    resp = requests.post(url, json=payload)
    if not resp.ok:
        raise RuntimeError(f"HTTP {resp.status_code}: {resp.text}")
    return resp.json()


# ─────────────────────────────────────────────────────────────
# MCP → smolagents Tool 매핑 (툴 이름은 Python identifier로 단순화)
# ─────────────────────────────────────────────────────────────

class CalcAddTool(Tool):
    name = "calc_add"
    description = "두 수를 더하는 MCP calc_add 호출"
    inputs = {
        "a": {"type": "number", "description": "첫 번째 숫자"},
        "b": {"type": "number", "description": "두 번째 숫자"},
    }
    output_type = "object"

    def forward(self, a: float, b: float):
        # ★ 여기: calc_add (언더스코어)
        return call_tool(BASE, "calc_add", {"a": a, "b": b})


class FxConvertTool(Tool):
    name = "fx_convert"
    description = "USD → KRW 환산 MCP fx_convert 호출"
    inputs = {
        "amount_usd": {"type": "number", "description": "환전할 USD 금액"},
    }
    output_type = "object"

    def forward(self, amount_usd: float):
        # ★ 여기: fx_convert
        return call_tool(BASE, "fx_convert", {"amount_usd": amount_usd})


class KbSearchTool(Tool):
    name = "kb_search"
    description = "코퍼스 검색 MCP kb_search 호출"
    inputs = {
        "query": {"type": "string", "description": "검색할 쿼리 문자열"},
        "top_k": {
            "type": "integer",
            "description": "반환할 최대 결과 수 (기본값 3)",
            "nullable": True,
        },
    }
    output_type = "object"

    def forward(self, query: str, top_k: int = 3):
        return call_tool(BASE, "kb_search", {"query": query, "top_k": top_k})


tools = [CalcAddTool(), FxConvertTool(), KbSearchTool()]
print("🔧 등록된 툴:", [t.name for t in tools])


# api_key = os.getenv("OPENAI_API_KEY")
# if not api_key:
#     raise SystemExit("⚠️ OPENAI_API_KEY 환경 변수가 없습니다!")

model = OpenAIModel(
    model_id="gpt-4o-mini",
    # api_key=api_key,
)

agent = ToolCallingAgent(
    tools=tools,
    model=model,
)

# ─────────────────────────────────────────────────────────────
# 실행 예시
# ─────────────────────────────────────────────────────────────

print("\n--- 실행 #1: 환전 ---")
print(agent.run("USD 100을 KRW로 바꿔줘."))

# print("\n--- 실행 #2: 덧셈 ---")
# print(agent.run("3과 7을 더해줘."))

# print("\n--- 실행 #3: 검색 ---")
# print(agent.run("HBM 관련 정보를 코퍼스에서 찾아 요약해줘."))


🔧 등록된 툴: ['calc_add', 'fx_convert', 'kb_search']

--- 실행 #1: 환전 ---


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ USD 100을 KRW로 바꿔줘.                                                                                         │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'fx_convert' with arguments: {'amount_usd': 100}                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: {'amount_krw': 135000.0, 'rate': 1350.0}

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '135000 KRW'}                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 135000 KRW

Final answer: 135000 KRW

[Step 2: Duration 0.72 seconds| Input tokens: 2,420 | Output tokens: 33]

135000 KRW
